# Actions to create buildings (aka Construction)
Populations take actions, one kind of action is the building. 


In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio



# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [2]:
from app.creators import universe
from app.functions import maths
from app.functions import configurations
from app.objects import time as t

from notebooks.helpers import test_queries

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.models as models

c = models.CosmosdbClient()

executing local windows deployment


**Note** that there is a field for `explanation` within the yaml in the doc

In [4]:
conf = configurations.get_building_configurations()
conf['building']['explanation']

[{'type': "all buildings have a 'type'/'label' of building"},
 {'name': "name_with_underscore. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'built_by': 'type of object that can build the building'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'changes': 'stats changed by the owner'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in `universal time units` that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'render_type': "guides the rendering in buildings.js. Available types: 'block'"}]

In [5]:
conf['building']['buildings'][0]

{'name': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'augments_resource': {'vegitable_foodstuff': 10},
 'changes': {'wealth': -0.01},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block'}

`Time` fetches all of the relevant actions

In [55]:
time = t.Time(c)
time.get_current_UTU()
time.get_global_actions()

## Adding a building action to a pop, for testing. 

Go to the app and create a building action. Then run this notebook. You should be able to see that a `construction` action has been created for that pop. 

This is done in `templates\app\ajax`, but for testing it's best to do it in the UI. 

In [35]:
actions_df = pd.DataFrame(time.actions)
actions_df

,action,job,agent
0,"{'type': 'individual_education', 'requires_att...","{'status': 'pending', 'username': 'BillmanLoca...","{'isIdle': 'false', 'name': 'Tiatairich Karich..."
1,"{'type': 'construction', 'comment': 'construct...","{'status': 'pending', 'username': 'BillmanLoca...","{'isIdle': 'false', 'name': 'Tiatairich Fa', '..."


from that dataframe we are going to get a list of action objects 

In [37]:
time.get_global_actions()

In [38]:
validActionCounter = 0
actions = [t.Action(c,actions_df.loc[i]) for i  in actions_df.index if actions_df.loc[i].action['type'] == 'construction']
actions

[< (Tiatairich Fa: 5478945973456) -takingAction:1017-> (construction) >]

In [39]:
a = actions[0]
a

< (Tiatairich Fa: 5478945973456) -takingAction:1017-> (construction) >

In [40]:
a.action['type']

'construction'

The agent within the action should have the information needed for the agent

In [41]:
a.agent['name']

'Tiatairich Fa'

# Resolving Actions

Note that the time of the action may not yet have come to pass. You'll have to run the dev engine to 'spin the clock' until the time to complete the action has come. 

In [56]:
time, a

(< time at: 2023-09-29T21:28:59.924901+00:00 UTU:1018 >,
 < (Tiatairich Fa: 5478945973456) -takingAction:1017-> (construction) >)

In [57]:
a.action.get('type') == 'construction'

True

In [58]:
b = [b for b in buildings_config['building']['buildings'] if b['name'] == a.action['building']][0]
b

{'name': 'solar_panel',
 'description': 'Generates wealth for all populations in this faction except the owner',
 'owned_by': 'pop',
 'effort': 3,
 'changes': {'wealth': -0.01},
 'faction_augments': {'wealth': 0.01},
 'requires_attr': {'industry': 0.51},
 'render_type': 'block'}

In [52]:
a.validate_action_time(time)

False

If this is false, you'll need to run the engine again to make sure that the action can happen. 

In [53]:
construction_actions = [a for a in actions if a.action['type'] == 'construction']
construction_actions

[< (Tiatairich Fa: 5478945973456) -takingAction:1017-> (construction) >]